In [ ]:
import sys
sys.path.append('/Users/jiangxiaoyu/Desktop/All Projects/Scalable_LVMOGP/')
import torch
from torch import Tensor
import numpy as np
import yaml
from code_blocks.our_models.Multi_ISVGP import Multi_Variational_IGP

In [ ]:
def load_model_and_likelihoods(multi_variational_igp, filename):
    state_dicts = torch.load(filename)
    for model, state_dict in zip(multi_variational_igp.models, state_dicts['models']):
        model.load_state_dict(state_dict)
    for likelihood, state_dict in zip(multi_variational_igp.likelihoods, state_dicts['likelihoods']):
        likelihood.load_state_dict(state_dict)

In [ ]:
curr_config = '/Users/jiangxiaoyu/Desktop/All Projects/Scalable_LVMOGP/configs/spatiotemp_IGP/Periodic_times_Scale_RBF/IndepSVGP_unfix.yaml'
with open(curr_config, 'r') as file:
        config = yaml.safe_load(file)

In [ ]:
MultiIGP = Multi_Variational_IGP(
        num_models = config['n_outputs'], 
        inducing_points = Tensor(np.linspace(config['init_inducing_input_LB'], config['init_inducing_input_UB'], config['n_inducing_input']).reshape(-1, 1)), 
        init_likelihood_noise = config['init_likelihood_noise'], 
        kernel_type = config['input_kernel_type'], 
        learn_inducing_locations= config['learn_inducing_locations_input'],
        input_dim = config['input_dim']
    )

In [ ]:
load_model_and_likelihoods(MultiIGP, '/Users/jiangxiaoyu/Desktop/All Projects/Scalable_LVMOGP/experiments_results/spatiotemp_IGP/Periodic_times_Scale_RBF/IndepSVGP_unfix/2024-02-19_12:24:14/MultiIGPs_models_and_likelihoods.pth')

In [ ]:
problemtic_model_1 = MultiIGP.get_model(740) # 333, 740, 885, 
problemtic_model_1_inducing_points = problemtic_model_1.variational_strategy.inducing_points.detach()

In [ ]:
random_points = torch.randn(20, 1)

In [ ]:
norm_model_1 = MultiIGP.get_model(500) # 500
norm_model_1_inducing_points = norm_model_1.variational_strategy.inducing_points.detach()

In [ ]:
for name, param in norm_model_1.covar_module_input.named_parameters():
    print(name)
    print(param)

In [ ]:
problemtic_model_1.covar_module_input.kernels[0].raw_lengthscale.data = torch.tensor([[-0.2725]])
problemtic_model_1.covar_module_input.kernels[0].raw_period_length.data = torch.tensor([[-0.9371]])
problemtic_model_1.covar_module_input.kernels[1].raw_outputscale.data = torch.tensor(0.6047)
problemtic_model_1.covar_module_input.kernels[1].base_kernel.raw_lengthscale.data = torch.tensor([[19.0558]])


problemtic_model_1.variational_strategy._variational_distribution = norm_model_1.variational_strategy._variational_distribution

In [ ]:
norm_model_1(random_points)

In [ ]:
for name, params in norm_model_1.named_parameters():
    print(name)

In [ ]:
# inducing_points = problemtic_model_1.variational_strategy.inducing_points.detach()
# problemtic_model_1.covar_module_input.kernels[0].raw_lengthscale.data = Tensor([[-0.5]])
# print(problemtic_model_1.covar_module_input.kernels[0].raw_lengthscale.data)

problemtic_model_1(random_points)